In [1]:
import json
import numpy as np
from bias_explorer.operations import report
from bias_explorer.utils import system, dataloader
import glob

In [2]:
predictions_path = "./data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/"

In [3]:
def get_topk_preds(preds_path):
    preds_list = glob.glob(predictions_path + 'top*.csv')
    k_preds = {}
    for pred in preds_list:
        fname = pred.split("/")[-1]
        name_parts = fname.split("_")
        if len(name_parts) > 2:
            k = name_parts[1]
        else:
            k = 1
        k_preds[k] = pred
    return k_preds

In [4]:
print(get_topk_preds(predictions_path))

{1: './data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/top_synms.csv', '2': './data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/top_2_synms.csv'}


In [8]:
avg_path = predictions_path + "sum_synms.csv"
mode_name = "Avg Sum"
metric_name = "accuracy"
sum_df = dataloader.load_df(avg_path)

In [20]:
def get_col_list(df, metric_name):
    if metric_name == "balanced_accuracy":
        col_list = list(df.drop(
            columns=['file', 'gender_preds', 'gender']).keys())
    else:
        col_list = list(df.drop(columns=['file', 'gender_preds']).keys())
        col_list = system.list_item_swap(col_list, 'age', 'gender')
    col_list = system.list_item_swap(col_list, 'age', 'race')
    return col_list

In [23]:
def get_uniques(df, col_list):
    uniques = []
    for col in col_list:
        col_items = list(df[col].unique())
        if col == "age":
            col_items = system.fix_age_order(col_items)
        for unique in col_items:
            if col == "age" and unique == "00-02":
                unique = "0-2"
            elif col == "age" and unique == "03-09":
                unique = "3-9"
            uniques.append(unique)
    return uniques

In [24]:
report_dict = {
    'Mode': [],
    metric_name: [],
}

col_list = get_col_list(sum_df, metric_name)
uniques = get_uniques(sum_df, col_list)
print(uniques)
print(col_list)

['Male', 'Female', 'East Asian', 'White', 'Latino_Hispanic', 'Southeast Asian', 'Black', 'Indian', 'Middle Eastern', '0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', 'more than 70']
['gender', 'race', 'age']


In [14]:
# gen_dict_report
def gen_dict_report(df, mode_name, metric_name):
    metric_func = report.metric_loader(metric_name)
    rep_dict = {}
    rep_dict['Mode'] = mode_name
    rep_dict[metric_name] = report.gender_eval(df, metric_func)
    col_list = get_col_list(df, metric_name)
    uniques = get_uniques(df, col_list)
    for col in col_list:
        for unique in uniques:
            col_df = report.filter_df(df, col, unique)
            col_acc = report.gender_eval(col_df, metric_func)
            rep_dict[unique] = col_acc
    return rep_dict

In [15]:
rep_dict = gen_dict_report(sum_df, 'Avg Sum', 'accuracy')

In [16]:
rep_dict

{'Mode': 'Avg Sum',
 'accuracy': 0.9582,
 'Male': 0.9356,
 'Female': 0.9835,
 'East Asian': 0.949,
 'White': 0.965,
 'Latino_Hispanic': 0.9698,
 'Southeast Asian': 0.9428,
 'Black': 0.9319,
 'Indian': 0.969,
 'Middle Eastern': 0.981,
 '0-2': 0.7085,
 '3-9': 0.8975,
 '10-19': 0.9441,
 '20-29': 0.9688,
 '30-39': 0.9845,
 '40-49': 0.9815,
 '50-59': 0.9724,
 '60-69': 0.9907,
 'more than 70': 0.9492}